# Übung zu Vorlesung Geodatenvisualisierung

11.12.2025



**Übungsaufgaben** zu folgenden Themen:

1. Projektionen verstehen (CRS / EPSG)
2. Statische Choroplethenkarte
3. Punktdaten & Overplotting mit Hexbins (Matplotlib)
4. Interaktive Karten mit Folium
5. Marktstammdatenregister (MaStR) mit Plotly & Hexbin-Aggregation

Zu jeder Aufgabe gibt es:
- eine **Aufgabenformulierung**,
- **Hinweise zur Lösung**,
- **Musterlösung** wird zum Download bereitgestellt.

In [ ]:
# Bibliotheken laden
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotnine import *

import folium
from folium.plugins import HeatMap

import plotly.express as px

import json
import geodatasets
import numpy as np

# Definiere das Plot-Layout für saubere Matplotlib-Grafiken
plt.style.use('default')


plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10

print('Bibliotheken geladen.')

In [ ]:
# URL zur Natural Earth Datensatz (1:110m, Cultural Vectors)
# Dies ist die stabilste Methode, da sie keine Paketabhängigkeiten hat.
NE_URL = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_0_countries.geojson"

try:
    # Laden der Weltkarte direkt über die URL
    world = gpd.read_file(NE_URL)
    print("Weltkarte erfolgreich über direkte URL geladen.")
except Exception as e:
    print(f"Fehler: Weltkarte konnte nicht über URL geladen werden. Fehler: {e}")
    # Der Fehler sollte hier nicht mehr auftreten.
    raise

# Filtern für eine saubere Weltkarte
# Hinweis: Die Spaltennamen im Natural Earth GeoJSON können leicht abweichen.
# Wir verwenden 'POP_EST' und 'SOVEREIGNT' für die Filterung.
#world_data = world[(world.SOVEREIGNT!="Antarctica")]
germany = world[world.SOVEREIGNT == 'Germany'].copy() 

# --- Anpassung der Spaltennamen für die Visualisierung ---
# Da wir 'pop_est' und 'name' in den Visualisierungen verwenden, stellen wir sicher,
# dass die Spalten entsprechend benannt sind.
world_data = world.rename(columns={'POP_EST': 'pop_est', 'SOVEREIGNT': 'name'})
germany = germany.rename(columns={'POP_EST': 'pop_est', 'SOVEREIGNT': 'name'})

### Aufgabe 1 

Erstelle eine Choroplethenkarte mit der Bevölkerung je Land

- Hinweis: Verwende den Natural Earth Datensatz für Ländergrenzen und die Spalte `POP_EST` für die Bevölkerung.
- Verwende GeoPandas zum Laden und Plotten der Daten.

In [ ]:
# your code here

### Aufgabe 2

Erstelle eine Chloroplethenkarte mit statistischer Darstellung (z.B. Balken, Kreis) nach Bevölkerungsgröße.

- Hinweis: Verwende den Natural Earth Datensatz für Ländergrenzen und die Spalte `POP_EST` für die Bevölkerung.
- Verwende `geopandas` und `matplotlib` für die Visualisierung.


In [ ]:
# your code here

### Aufgabe 3

Erstelle eine Chloroplethenkarte der Länder von Asien zu GDP und GDP/Capita mit Plotnine Syntax

- Hinweis: Verwende den Natural Earth Datensatz für Ländergrenzen und `continent`.
- Beachte: Normiere die Kennzahl mit Bevölkerung.
- Verwende `plotnine` für die Visualisierung.
- Was fällt zwischen den beiden Karten auf? Worauf wirkt sich die Normierung aus?

In [ ]:
# Filtern und Kennzahl berechnen
world_data['gdp_cap'] = world_data['GDP_MD'] / world_data['pop_est']
asia = world_data[world_data['CONTINENT'] == "Asia"]


In [ ]:
# GDP absolut in Asien visualisieren
# your code here

In [ ]:
# GDP pro Kopf in Asien visualisieren
# your code here

### Aufgabe 4

Erstelle eine interaktive Choroplethenkarte mit Plotly, die die Bevölkerung je Land darstellt.

- Hinweis: Verwende den Natural Earth Datensatz für Ländergrenzen und die Spalte `POP_EST` für die Bevölkerung.
- Verwende `plotly.express` für die interaktive Visualisierung.
- Beachte: plotly benötigt die Geometriedaten im GeoJSON-Format.

In [ ]:
# Plotly benötigt die Geometriedaten im GeoJSON-Format
world_geojson = json.loads(world_data.to_json())

# your code here

### Bonus

Spiele mit den Optionen und ergänze ein Grid-Overlay oder Tooltips.

- Hinweis: Durchsuche die Plotly-Dokumentation für weitere Anpassungsmöglichkeiten und Features wie Grid-Overlay oder Tooltips.

In [ ]:
# your code here

### Aufgabe 5 

Lerne den Unterschied zwischen unterschiedlichen Projektionen kennen ([hier](https://plotly.com/python/reference/layout/geo/#layout-geo-projection-type)). Konkret sollst Du ausprobieren 

- projection_type='natural earth'
- projection_type='orthographic'
- projection_type='mercator'
- projection_type='transverse mercator'
- projection_type="equirectangular"
- projection_type="azimuthal equal area"

Hinweise: 
- Verwende immer die gleiche von Dir ausgewählte Plotly Visualisierung (gerne auch aus vorheriger Übung übernehmen) 
- Was fällt Dir aus den Vergleichen auf? Gerne auch mal auf bestimmte Länder fokussieren und die Unterschiede der Projektionen vergleichen.

In [ ]:

# Plotly benötigt die Geometriedaten im GeoJSON-Format
world_geojson = json.loads(world_data.to_json())

# Definiere die Projektion (Plotly erwartet z.B. "natural earth")
projection = "natural earth"

# Plotly Choroplethenkarte - Beispiel
fig_plotly_choro = px.choropleth(
    world_data,
    geojson=world_geojson,
    locations='name',  # Die Spalte, die den Join-Key enthält 
    featureidkey="properties.name", # Pfad zum Join-Key in der GeoJSON-Datei
    color='pop_est',   # Die Spalte, die zur Einfärbung verwendet wird  
    hover_name="name", # Text, der beim Hovern angezeigt wird
    hover_data={'pop_est': True},
    color_continuous_scale=px.colors.sequential.Plasma,
    projection=projection,
    title=f"Interaktive Weltbevölkerung ({projection})"
)

# Kartenansicht anpassen
fig_plotly_choro.update_geos(fitbounds="locations", visible=False)
fig_plotly_choro.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

fig_plotly_choro.show()

In [ ]:
# your code here

### Aufgabe 6

Erstelle eine interaktive Folium Karte und setze einen Marker auf Deine Wohnort-Koordinaten. 
Mit dem Marker soll ein Popup erscheinen, das Deinen Namen und Deine Adresse enthält.


Hinweis: 
- Die Wohnort-Koordinaten (Geographische Koordinaten Längen/Breitengrad) kannst Du z.B. über Google Maps ermitteln. 
- Verwende die `folium` Bibliothek für die Erstellung der Karte und das Hinzufügen des Markers mit Popup.
- Experimentiere mit verschiedenen Basiskarten. Recherchiere dazu die verschiedenen `tiles`-Optionen in der Folium-Dokumentation.
- Speichere die erstellte Karte als HTML-Datei, damit Du sie in einem Webbrowser anzeigen kannst.

In [ ]:
# Karte mit OSM-Kacheln zentriert auf Deutschland
map = folium.Map(
    location=[51.0, 10.0],   # ungefährer Mittelpunkt von Deutschland 
    zoom_start=6,            # Startzoom
    tiles="OpenStreetMap"    # OSM als Basiskarte
)
map


In [ ]:
# Karte mit OSM-Kacheln zentriert auf Deutschland
mymap = folium.Map(
    location=[51.0, 10.0],   # Mittelpunkt
    zoom_start=3,            # Startzoom
    tiles="OpenStreetMap"    # OSM als Basiskarte
)

# Beispiel-Punkt (z.B. Nürnberg)
folium.Marker(
    location=[49.4521, 11.0767],
    popup=folium.Popup(html="<b>Nürnberg</b><br>Lat: 49.4521<br>Lon: 11.0767", max_width=200)
).add_to(mymap)

mymap

# Exportiere die Karte als HTML-Datei
path = "../data"
mymap.save(f"{path}/folium_mymap.html")


In [ ]:
# your code here

### Aufgabe 7

Erstelle eine Karte in Folium oder Plotly, die die Standorte (Punkte) von Kraftwerken in Deutschland aus dem Marktstammdatenregister (MaStR) visualisiert. 
- Hinweis: Lade die Kraftwerksdaten aus dem MaStR-Datensatz und filtere die Daten für Deutschland.
- Verwende `folium` oder `plotly` für die Visualisierung.

In [ ]:
# Pfad zur GeoJSON-Datei mit Gemeindegrenzen in Deutschland (bitte individuell anpassen) - Quelle https://github.com/opendatalab-de/simple-geodata-selector/blob/master/src/data/gemeinden_sim20.geojson herunterladen 
path = r"C:\gemeinden_simplify20.geojson"

# Lade GeoJSON-Geometrien von Pfad
gemeinden = gpd.read_file(path)

# Konvertiere Datetime-/Timestamp-Spalten zu Strings, damit Folium/JSON keine Fehler wirft
import pandas as _pd
for col in gemeinden.columns:
	if _pd.api.types.is_datetime64_any_dtype(gemeinden[col]) or _pd.api.types.is_timedelta64_dtype(gemeinden[col]):
		gemeinden[col] = gemeinden[col].astype(str)

# Map: gemeinden als Overlay über Folium OSM Basemap
m = folium.Map(location=[51.0, 10.0], zoom_start=6, tiles="OpenStreetMap")

# Verwende das GeoJSON-Interface der GeoDataFrame (nun ohne Timestamp-Objekte)
folium.GeoJson(gemeinden.__geo_interface__, name="gemeinden").add_to(m)

m

In [ ]:
# Lade Marktstammdatenregister - individuell anpassen
mastr_path = r"desc_solar_de.csv"

mastr = pd.read_csv(mastr_path, sep=',')
mastr.info()

mastr.head()

### Aufgabe 8 

Plotte zunächst die Anlagen in Bayern als Punkte karte. Erstelle dann eine weitere Heatmap-Karte der Kraftwerksstandorte (lat, lon aus mastr) in Bayern mit Plotly.

- Hinweis: Verwende die Kraftwerksdaten aus dem MaStR-Datensatz.
- Verwende `plotly.express` für die Erstellung der Hexbin-Karte.

In [ ]:
# Plotte alle Anlagen in Bayern (lat lon) auf einer folium Karte 
mastr_by = mastr[mastr['state'] == 'Bayern'].copy()

# Karte mit OSM-Kacheln zentriert auf Bayern
m_bayern = folium.Map(
    location=[48.0, 11.0],   # Mittelpunkt
    zoom_start=7,            # Startzoom
    tiles="OpenStreetMap"    # OSM als Basiskarte
    )
# Füge alle Anlagen als Punkte hinzu
for idx, row in mastr_by.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=folium.Popup(html=f"<b>Anlage ID:</b> {row['unit_number']}<br><b>Leistung (kW):</b> {row['unit_gross_capacity']}", max_width=200)
    ).add_to(m_bayern)

    m_bayern

In [ ]:
# your code here

### Aufgabe 8 

Visualisiere auf der Basis von Gemeinden in Bayern (Geojson File) die Nettokapazität der im MaStR erfassten Anlagen pro Kopf. Hierfür musst Du einen Point-in-Polygon Spatial Join durchführen, um die Anlagen den Gemeinden zuzuordnen.  

Hinweise: 
- Nutze die Geojson Datei laden 
- Pro Kopf Berechnung: Nettokapazität der Anlagen in der Gemeinde / Einwohnerzahl der Gemeinde -> Nutze dafür die Bevölkerung aus den Gemeindaten (aus dem Array "destatis"). Funktion siehe unten
- Für eine bessere Übersichtlichkeit verwende eine diskrete Klassifizierung (z.B. Quintile) der Nettokapazität pro Kopf. 
- Verwende `plotly.express` für die interaktive Choroplethenkarte mit Mapbox als Basiskarte.
- Achte auf die Zentrierung und den Zoom-Level der Karte, damit Bayern gut sichtbar ist.






In [ ]:
# Extrahiere die Bevölkerungszahl aus der Spalte 'destatis'.
# Die Spalte kann bereits dict-Objekte oder JSON-Strings enthalten -> parsen.

def _extract_population(d):
	if pd.isna(d):
		return None
	if isinstance(d, dict):
		return d.get("population")
	if isinstance(d, str):
		try:
			obj = json.loads(d)
			if isinstance(obj, dict):
				return obj.get("population")
		except Exception:
			return None
	return None

gemeinden["population"] = gemeinden["destatis"].apply(_extract_population)
gemeinden["population"] = pd.to_numeric(gemeinden["population"], errors="coerce")
gemeinden["population"] = gemeinden["population"] /1000  # Bevölkerung in Tausend

In [ ]:
# Vor dem spatial join als GeoDataFrame speichern

mastr_by_gdf = gpd.GeoDataFrame(
    mastr_by,
    geometry=gpd.points_from_xy(mastr_by['lon'], mastr_by['lat']),
    crs='EPSG:4326'
)

In [ ]:
# Spatial Join: Aggregate MaStR Leistung pro Gemeinde (points within polygons)
mastr_by_capacity_gdf = gpd.sjoin(mastr_by_gdf, gemeinden, how='inner', predicate='within').groupby('AGS_0').agg({
    'unit_gross_capacity': 'sum',   # Summe der Leistung pro Gemeinde
    'lat': 'first',                 # Beispiel: erste Latitude pro Gemeinde
    'lon': 'first',                 # Beispiel: erste Longitude pro Gemeinde
    'locality' : 'first',           # Beispiel: Name pro Gemeinde
    'population': 'first'           # Beispiel: Bevölkerung pro Gemeinde
    }).reset_index()

# pro Kopf der Gemeinde berechnen
mastr_by_capacity_gdf['unit_gross_capacity_cap'] = mastr_by_capacity_gdf['unit_gross_capacity'] / mastr_by_capacity_gdf['population'] 


In [ ]:
# Auf basis von mastr_by_capacity_gdf eine plotly choroplethenkarte erstellen, die die unit_gross_capacity_cap pro gemeinde in bayern darstellt. Nutze dazu die gemeinden GeoDataFrame als geojson.

# your code here

In [ ]:
# 1) Gemeinden nach WGS84 projizieren
gemeinden_wgs = gemeinden.to_crs(epsg=4326)

# 2) Nur die für GeoJSON/Plotly relevanten Spalten behalten
#    (hier: AGS_0 + geometry; alles andere fliegt raus, inkl. evtl. Timestamp-Spalten)
gemeinden_for_json = gemeinden_wgs[['AGS_0', 'geometry']].copy()

# 3) GeoJSON aus dem "abgespeckten" GeoDataFrame erstellen
gemeinden_geojson = json.loads(gemeinden_for_json.to_json())

# 4) IDs als String sicherstellen
mastr_by_capacity_gdf["AGS_0"] = mastr_by_capacity_gdf["AGS_0"].astype(str)
gemeinden_for_json["AGS_0"] = gemeinden_for_json["AGS_0"].astype(str)

# 5) Choroplethenkarte mit OSM-Baselayer
# your code here

In [ ]:
# Nach quintilen einfärben

# your code here